In [1]:
import pandas as pd

In [130]:
airline = pd.read_csv('airlines.csv')
alliance = pd.read_csv('alliances.csv')
route = pd.read_csv('routes.csv')
airport = pd.read_csv('airports-extended.csv', header=None, 
                      names=['id', 'airport_name', 'city', 'country', 'IATA', 'ICAO',
                             'latitude', 'longitude', 'altitude',
                             'timezone', 'daylight_saving_time', 'database_timezone', 'type', 'source'])

In [21]:
alliance.Member.value_counts()

Star Alliance    27
Skyteam          20
Oneworld         15
Name: Member, dtype: int64

In [38]:
df = airline.merge(alliance, how='right', on='IATA')
df = df[df.Name_x == df.Name_y]
df = df.merge(route, how='left', left_on='IATA', right_on='airline')

In [40]:
df.columns

Index(['Airline ID', 'Name_x', 'Alias', 'IATA', 'ICAO', 'Callsign', 'Country',
       'Active', 'Name_y', 'Member', 'airline', 'airline ID',
       ' source airport', ' source airport id', ' destination apirport',
       ' destination airport id', ' codeshare', ' stops', ' equipment'],
      dtype='object')

In [89]:
def compute_edge_list(df, group, column='Member'):
    df = df[df[column] == group]
    df1 = df[df[' source airport'] < df[' destination apirport']].\
        groupby([' source airport', ' destination apirport']).IATA.count().\
        reset_index().rename(columns={' source airport': 'airport1', ' destination apirport': 'airport2'})
    df2 = df[df[' source airport'] > df[' destination apirport']].\
        groupby([' destination apirport', ' source airport']).IATA.count().\
        reset_index().rename(columns={' destination apirport': 'airport1', ' source airport': 'airport2'})
    df3 = df1.merge(df2, how='outer', on=['airport1', 'airport2']).fillna(0)
    df3[group] = df3.IATA_x + df3.IATA_y
    return df3[['airport1', 'airport2', group]]

def get_all_edge_lists(df, column='Member'):
    edge_lists = None
    for group in df[column].unique():
        if edge_lists is None:
            edge_lists = compute_edge_list(df, group)
        else:
            edge_lists = edge_lists.merge(compute_edge_list(df, group), how='outer', on=['airport1', 'airport2']).fillna(0)
    return edge_lists

In [90]:
edge_lists = get_all_edge_lists(df)

In [124]:
edge_lists.rename(columns={'Star Alliance': 'StarAlliance'}).to_csv('airline_edges.csv', index=False)

In [131]:
nodes = pd.DataFrame(sorted(list(set(edge_lists.airport1) | set(edge_lists.airport2))), columns=['IATA'])

In [132]:
nodes = nodes.merge(airport, how='left', on='IATA')

In [133]:
nodes.to_csv('airline_nodes.csv', index=False)

In [134]:
edge_lists.Skyteam.sum()

10794.0